In [ ]:
# ==================== LOAD DATASETS ====================
print("\n" + "="*80)
print("LOADING DATASETS FROM KAGGLE")
print("="*80)

import glob

# Try Kaggle's native dataset access first (for Kaggle notebooks)
try:
    # On Kaggle, datasets are mounted at /kaggle/input/
    diabetes_paths = glob.glob('/kaggle/input/*/diabetes_binary_health_indicators_BRFSS2015.csv')
    adult_paths = glob.glob('/kaggle/input/*/adult.csv')
    
    if diabetes_paths and adult_paths:
        diabetes_csv = diabetes_paths[0]
        adult_csv = adult_paths[0]
        print("✓ Using Kaggle native dataset paths")
    else:
        raise FileNotFoundError("Datasets not found in /kaggle/input/")
        
except (FileNotFoundError, IndexError):
    # Fallback to kagglehub for local execution
    print("✓ Using kagglehub for dataset download")
    import kagglehub
    
    diabetes_path = kagglehub.dataset_download("alexteboul/diabetes-health-indicators-dataset")
    diabetes_csv = f"{diabetes_path}/diabetes_binary_health_indicators_BRFSS2015.csv"
    
    adult_path = kagglehub.dataset_download("uciml/adult-census-income")
    adult_csv = f"{adult_path}/adult.csv"

# Load datasets
df_diabetes = pd.read_csv(diabetes_csv)
print(f"✓ Diabetes dataset loaded: {df_diabetes.shape}")

df_adult = pd.read_csv(adult_csv)
print(f"✓ Adult dataset loaded: {df_adult.shape}")

In [ ]:
# ==================== PREPROCESS DATA ====================
print("\n" + "="*80)
print("PREPROCESSING DATA")
print("="*80)

# Diabetes
X_diabetes = df_diabetes.drop(columns=['Diabetes_binary']).values
y_diabetes = df_diabetes['Diabetes_binary'].values
print(f"✓ Diabetes - Features: {X_diabetes.shape}, Target: {y_diabetes.shape}")

# Adult
X_adult_df = df_adult.drop(columns=['income'])
y_adult = (df_adult['income'] == '>50K').astype(int).values

# Encode categorical features
categorical_cols = X_adult_df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    le = LabelEncoder()
    X_adult_df[col] = le.fit_transform(X_adult_df[col].astype(str))

# Convert to numpy array
X_adult = X_adult_df.values
print(f"✓ Adult - Features: {X_adult.shape}, Target: {y_adult.shape}")

# Verify both are numpy arrays
print(f"\n✓ Data types verified:")
print(f"  X_diabetes: {type(X_diabetes)}")
print(f"  X_adult: {type(X_adult)}")
print(f"  y_diabetes: {type(y_diabetes)}")
print(f"  y_adult: {type(y_adult)}")

In [ ]:
# ==================== MODEL ARCHITECTURES ====================

class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, output_size=2):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
    
    def forward(self, x):
        return self.linear(x)

class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_sizes=[128, 64], output_size=2, dropout_rate=0.3):
        super().__init__()
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, output_size))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

print("✓ Model architectures defined")

In [ ]:
# ==================== FEDERATED LEARNING UTILITIES ====================

def fedavg_aggregate(client_models, client_data_sizes):
    """FedAvg: Weighted average based on client data sizes"""
    total_size = sum(client_data_sizes)
    global_state = {}
    
    for key in client_models[0].state_dict().keys():
        global_state[key] = sum(
            client_models[i].state_dict()[key] * (client_data_sizes[i] / total_size)
            for i in range(len(client_models))
        )
    
    return global_state

def fedprox_aggregate(client_models, client_data_sizes):
    """FedProx: Similar to FedAvg"""
    return fedavg_aggregate(client_models, client_data_sizes)

def qfedavg_aggregate(client_models, client_losses, q=0.2):
    """q-FedAvg: Fairness-weighted aggregation"""
    lipschitz = [1.0 / (loss + 1e-10) for loss in client_losses]
    weights = [l ** q for l in lipschitz]
    total_weight = sum(weights)
    
    global_state = {}
    for key in client_models[0].state_dict().keys():
        global_state[key] = sum(
            client_models[i].state_dict()[key] * (weights[i] / total_weight)
            for i in range(len(client_models))
        )
    
    return global_state

def scaffold_aggregate(client_models, client_data_sizes):
    """SCAFFOLD: Simplified version"""
    return fedavg_aggregate(client_models, client_data_sizes)

# FedAdam state
fedadam_state = {'m': None, 'v': None, 't': 0}

def fedadam_aggregate(client_models, client_data_sizes, beta1=0.9, beta2=0.999, eta=0.01, tau=1e-3):
    """FedAdam: Adaptive federated optimization"""
    total_size = sum(client_data_sizes)
    avg_state = {}
    
    # Get device from first model
    device = next(client_models[0].parameters()).device
    
    for key in client_models[0].state_dict().keys():
        avg_state[key] = sum(
            client_models[i].state_dict()[key] * (client_data_sizes[i] / total_size)
            for i in range(len(client_models))
        )
    
    if fedadam_state['m'] is None:
        fedadam_state['m'] = {key: torch.zeros_like(val).to(device) for key, val in avg_state.items()}
        fedadam_state['v'] = {key: torch.zeros_like(val).to(device) for key, val in avg_state.items()}
    
    fedadam_state['t'] += 1
    global_state = {}
    
    for key in avg_state.keys():
        delta = avg_state[key]
        # Ensure state tensors are on the same device
        fedadam_state['m'][key] = fedadam_state['m'][key].to(device)
        fedadam_state['v'][key] = fedadam_state['v'][key].to(device)
        
        fedadam_state['m'][key] = beta1 * fedadam_state['m'][key] + (1 - beta1) * delta
        fedadam_state['v'][key] = beta2 * fedadam_state['v'][key] + (1 - beta2) * (delta ** 2)
        
        m_hat = fedadam_state['m'][key] / (1 - beta1 ** fedadam_state['t'])
        v_hat = fedadam_state['v'][key] / (1 - beta2 ** fedadam_state['t'])
        
        global_state[key] = avg_state[key] + eta * m_hat / (torch.sqrt(v_hat) + tau)
    
    return global_state

def train_fl_client(client_model, train_loader, global_model=None, epochs=5, lr=0.001, mu=0.01, use_proximal=False):
    """Train a single FL client"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    client_model = client_model.to(device)
    
    # Move global model to same device if using FedProx
    if use_proximal and global_model is not None:
        global_model = global_model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(client_model.parameters(), lr=lr)
    
    client_model.train()
    total_loss = 0.0
    
    for epoch in range(epochs):
        epoch_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = client_model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # FedProx proximal term
            if use_proximal and global_model is not None:
                proximal_term = 0.0
                for w, w_t in zip(client_model.parameters(), global_model.parameters()):
                    proximal_term += (w - w_t).norm(2)
                loss += (mu / 2) * proximal_term
            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        
        total_loss += epoch_loss / len(train_loader)
    
    return client_model, total_loss / epochs

def distribute_data_to_clients(X, y, num_clients, batch_size):
    """Distribute data IID to clients"""
    dataset = TensorDataset(torch.FloatTensor(X), torch.LongTensor(y))
    
    total_size = len(dataset)
    client_sizes = [total_size // num_clients] * num_clients
    client_sizes[-1] += total_size % num_clients
    
    indices = torch.randperm(total_size).tolist()
    client_loaders = []
    start_idx = 0
    
    for size in client_sizes:
        client_indices = indices[start_idx:start_idx + size]
        client_dataset = Subset(dataset, client_indices)
        client_loader = DataLoader(client_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        client_loaders.append(client_loader)
        start_idx += size
    
    return client_loaders, client_sizes

def evaluate_model(model, X, y):
    """Evaluate model and return accuracy, f1"""
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    
    X_tensor = torch.FloatTensor(X).to(device)
    
    with torch.no_grad():
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs, 1)
    
    accuracy = accuracy_score(y, predicted.cpu().numpy())
    f1 = f1_score(y, predicted.cpu().numpy(), average='weighted', zero_division=0)
    
    return accuracy, f1

print("✓ FL utilities defined")

In [ ]:
# ==================== PHASE 1: FEDERATED LEARNING - 5-FOLD CV × 5 RUNS ====================
print("\n" + "="*80)
print("PHASE 1: FEDERATED LEARNING WITH CROSS-VALIDATION")
print("="*80)

# Create models directory (works on both Windows and Linux/Kaggle)
try:
    # Try Kaggle/Linux path first
    models_dir = "/kaggle/working/models_fl_continue"
    os.makedirs(models_dir, exist_ok=True)
except:
    # Fallback to Windows path for local execution
    models_dir = os.path.join(r"c:\Users\almir\ai-privacy\backend", "models_fl_continue")
    os.makedirs(models_dir, exist_ok=True)

print(f"\nModels will be saved to: {models_dir}")
print(f"Total expected evaluations per config: {len(RANDOM_SEEDS)} seeds × {N_SPLITS} folds = {len(RANDOM_SEEDS) * N_SPLITS}")

# Safety check: Ensure data is numpy arrays, not DataFrames
if isinstance(X_diabetes, pd.DataFrame):
    X_diabetes = X_diabetes.values
if isinstance(y_diabetes, pd.Series):
    y_diabetes = y_diabetes.values
if isinstance(X_adult, pd.DataFrame):
    X_adult = X_adult.values
if isinstance(y_adult, pd.Series):
    y_adult = y_adult.values

print(f"\n✓ Data format verified:")
print(f"  X_diabetes: {type(X_diabetes)} shape {X_diabetes.shape}")
print(f"  X_adult: {type(X_adult)} shape {X_adult.shape}")

DATASETS = {'diabetes': (X_diabetes, y_diabetes), 'adult': (X_adult, y_adult)}
MODEL_TYPES = ['LR', 'FNN']
AGGREGATION_METHODS = ['FedAvg', 'FedProx', 'q-FedAvg', 'SCAFFOLD', 'FedAdam']

# Initialize checkpoint - Resume after FedProx (skip FedAvg and FedProx)
fl_results = {
    'diabetes_LR_FedAvg': {'dataset': 'diabetes', 'model': 'LR', 'aggregation': 'FedAvg'},
    'diabetes_LR_FedProx': {'dataset': 'diabetes', 'model': 'LR', 'aggregation': 'FedProx'}
}
resume_from = {'dataset': 'diabetes', 'model': 'LR', 'aggregation': 'q-FedAvg', 'found': False}
print("\n✓ Checkpoint initialized - Resuming from Diabetes LR q-FedAvg")

checkpoint_path = os.path.join(models_dir, 'fl_checkpoint.json')

for dataset_name, (X_data, y_data) in DATASETS.items():
    print("\n" + "-"*80)
    print(f"Dataset: {dataset_name.upper()}")
    print("-"*80)
    
    for model_type in MODEL_TYPES:
        print(f"\n  Model: {model_type}")
        
        for agg_method in AGGREGATION_METHODS:
            # Skip if already completed (checkpoint resume)
            config_key = f"{dataset_name}_{model_type}_{agg_method}"
            if config_key in fl_results:
                print(f"\n    Aggregation: {agg_method} - ✓ SKIPPED (already completed)")
                continue
            
            # Skip if before resume point
            if resume_from is not None:
                if (dataset_name != resume_from['dataset'] or 
                    model_type != resume_from['model'] or 
                    agg_method != resume_from['aggregation']):
                    if resume_from.get('found', False):
                        pass  # Already past resume point, continue normally
                    else:
                        print(f"\n    Aggregation: {agg_method} - ⏭️  SKIPPING (before resume point)")
                        continue
                else:
                    resume_from['found'] = True
                    print(f"\n    Aggregation: {agg_method} - 🔄 RESUMING FROM HERE")
            else:
                print(f"\n    Aggregation: {agg_method}")
            
            all_accuracies = []
            all_f1s = []
            
            # Reset FedAdam state
            fedadam_state['m'] = None
            fedadam_state['v'] = None
            fedadam_state['t'] = 0
            
            for run_idx, seed in enumerate(RANDOM_SEEDS):
                # K-Fold split
                skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=seed)
                
                for fold_idx, (train_idx, val_idx) in enumerate(skf.split(X_data, y_data)):
                    X_train, X_val = X_data[train_idx], X_data[val_idx]
                    y_train, y_val = y_data[train_idx], y_data[val_idx]
                    
                    # Scale features
                    scaler = StandardScaler()
                    X_train_scaled = scaler.fit_transform(X_train)
                    X_val_scaled = scaler.transform(X_val)
                    
                    # Distribute data to clients
                    client_loaders, client_sizes = distribute_data_to_clients(
                        X_train_scaled, y_train, NUM_CLIENTS, BATCH_SIZE
                    )
                    
                    # Initialize global model
                    input_size = X_train_scaled.shape[1]
                    if model_type == 'LR':
                        global_model = LogisticRegressionModel(input_size, output_size=2)
                    else:
                        global_model = FeedforwardNN(input_size, hidden_sizes=[128, 64], output_size=2)
                    
                    # FL Training
                    for round_num in range(GLOBAL_ROUNDS):
                        client_models = []
                        client_losses = []
                        
                        for client_id in range(NUM_CLIENTS):
                            client_model = deepcopy(global_model)
                            use_proximal = (agg_method == 'FedProx')
                            
                            trained_model, loss = train_fl_client(
                                client_model, client_loaders[client_id],
                                global_model=global_model if use_proximal else None,
                                epochs=LOCAL_EPOCHS, lr=LEARNING_RATE, use_proximal=use_proximal
                            )
                            
                            client_models.append(trained_model)
                            client_losses.append(loss)
                        
                        # Aggregate
                        if agg_method == 'FedAvg':
                            global_state = fedavg_aggregate(client_models, client_sizes)
                        elif agg_method == 'FedProx':
                            global_state = fedprox_aggregate(client_models, client_sizes)
                        elif agg_method == 'q-FedAvg':
                            global_state = qfedavg_aggregate(client_models, client_losses)
                        elif agg_method == 'SCAFFOLD':
                            global_state = scaffold_aggregate(client_models, client_sizes)
                        elif agg_method == 'FedAdam':
                            global_state = fedadam_aggregate(client_models, client_sizes)
                        
                        global_model.load_state_dict(global_state)
                    
                    # Evaluate
                    accuracy, f1 = evaluate_model(global_model, X_val_scaled, y_val)
                    all_accuracies.append(accuracy)
                    all_f1s.append(f1)
                    
                    if fold_idx == N_SPLITS - 1:
                        print(f"      Run {run_idx + 1}, Fold {fold_idx + 1}: Acc={accuracy:.4f}")
            
            # Statistics
            acc_mean = np.mean(all_accuracies)
            acc_std = np.std(all_accuracies, ddof=1)
            acc_min = np.min(all_accuracies)
            acc_max = np.max(all_accuracies)
            f1_mean = np.mean(all_f1s)
            f1_std = np.std(all_f1s, ddof=1)
            
            config_key = f"{dataset_name}_{model_type}_{agg_method}"
            fl_results[config_key] = {
                'dataset': dataset_name,
                'model': model_type,
                'aggregation': agg_method,
                'accuracy': {'mean': acc_mean, 'std': acc_std, 'min': acc_min, 'max': acc_max},
                'f1': {'mean': f1_mean, 'std': f1_std},
                'all_accuracies': all_accuracies,
                'all_f1s': all_f1s
            }
            
            print(f"\n    ✓ {agg_method} Results:")
            print(f"      Accuracy: {acc_mean*100:.2f}% ± {acc_std*100:.2f}% (range: {acc_min*100:.2f}% - {acc_max*100:.2f}%)")
            print(f"      F1-Score: {f1_mean*100:.2f}% ± {f1_std*100:.2f}%")
            
            # Save checkpoint after each aggregation
            checkpoint = {
                'fl_results': fl_results,
                'resume_from': {
                    'dataset': dataset_name,
                    'model': model_type,
                    'aggregation': agg_method,
                    'found': True
                },
                'timestamp': datetime.now().isoformat()
            }
            with open(checkpoint_path, 'w') as f:
                json.dump(checkpoint, f, indent=2, default=lambda x: float(x) if isinstance(x, np.floating) else x)
            print(f"      💾 Checkpoint saved")

print("\n" + "="*80)
print("FEDERATED LEARNING PHASE COMPLETE")
print("="*80)

In [ ]:
# ==================== PHASE 2: STATISTICAL ANALYSIS & COMPARISON ====================
print("\n" + "="*80)
print("PHASE 2: STATISTICAL ANALYSIS")
print("="*80)

# Load baseline results from crossvalidations.ipynb
try:
    # Try Kaggle/Linux path first
    baseline_path = "/kaggle/input/ai-privacy-baseline-results/research_results.json"
    with open(baseline_path, 'r') as f:
        baseline_data = json.load(f)
except:
    # Fallback to Windows path for local execution
    baseline_path = os.path.join(r"c:\Users\almir\ai-privacy\backend", "models_research", "research_results.json")
    with open(baseline_path, 'r') as f:
        baseline_data = json.load(f)

baseline_results = {}
for dataset in ['diabetes', 'adult']:
    for model in ['LR', 'FNN']:
        key = f"{dataset}_{model}"
        baseline_results[key] = {
            'accuracy': baseline_data['baseline_results'][dataset][model]['accuracy']['mean'],
            'all_accuracies': baseline_data['baseline_results'][dataset][model]['all_accuracies']
        }

print("\n✓ Baseline results loaded")

# FL vs Baseline comparisons
print("\n" + "="*80)
print("FEDERATED LEARNING vs BASELINE - Statistical Tests")
print("="*80)

fl_comparison = []
for config_key, fl_data in fl_results.items():
    # Skip placeholder entries (those without accuracy data)
    if 'accuracy' not in fl_data or 'all_accuracies' not in fl_data:
        continue
    
    baseline_key = f"{fl_data['dataset']}_{fl_data['model']}"
    baseline_acc = baseline_results[baseline_key]['accuracy']
    baseline_all = baseline_results[baseline_key]['all_accuracies']
    
    fl_acc = fl_data['accuracy']['mean']
    fl_all = fl_data['all_accuracies']
    accuracy_loss = baseline_acc - fl_acc
    
    # T-test
    t_stat, p_value = stats.ttest_ind(baseline_all, fl_all)
    
    fl_comparison.append({
        'Dataset': fl_data['dataset'],
        'Model': fl_data['model'],
        'Aggregation': fl_data['aggregation'],
        'FL_Accuracy': fl_acc * 100,
        'FL_Std': fl_data['accuracy']['std'] * 100,
        'Baseline': baseline_acc * 100,
        'Accuracy_Loss': accuracy_loss * 100,
        't_statistic': t_stat,
        'p_value': p_value,
        'Significant': 'Yes' if p_value < 0.05 else 'No'
    })

fl_comparison_df = pd.DataFrame(fl_comparison)
print("\n" + fl_comparison_df.to_string(index=False))

In [ ]:
# ==================== PHASE 3: SAVE RESULTS ====================
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# Save comprehensive JSON
results_json = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'random_seeds': RANDOM_SEEDS,
        'n_splits': N_SPLITS,
        'total_evaluations': len(RANDOM_SEEDS) * N_SPLITS,
        'note': 'Resumed after FedProx - contains q-FedAvg, SCAFFOLD, FedAdam for all configs'
    },
    'federated_learning': fl_results,
    'baseline_reference': baseline_results
}

json_path = os.path.join(models_dir, 'fl_continue_results.json')
with open(json_path, 'w') as f:
    json.dump(results_json, f, indent=2, default=lambda x: float(x) if isinstance(x, np.floating) else x)
print(f"✓ Saved: fl_continue_results.json")

# Save comparison CSV
fl_csv_path = os.path.join(models_dir, 'fl_vs_baseline.csv')
fl_comparison_df.to_csv(fl_csv_path, index=False)
print(f"✓ Saved: fl_vs_baseline.csv")

print("\n" + "="*80)
print("ALL RESULTS SAVED")
print("="*80)

In [ ]:
# ==================== PHASE 4: VISUALIZATIONS ====================
print("\n" + "="*80)
print("GENERATING VISUALIZATIONS")
print("="*80)

# FL Comparison Plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Federated Learning vs Baseline - 5-Fold CV × 5 Runs (Continued)', fontsize=16, fontweight='bold')

for idx, (dataset, model) in enumerate([('diabetes', 'LR'), ('diabetes', 'FNN'), ('adult', 'LR'), ('adult', 'FNN')]):
    ax = axes[idx // 2, idx % 2]
    
    # Filter data
    subset = fl_comparison_df[(fl_comparison_df['Dataset'] == dataset) & (fl_comparison_df['Model'] == model)]
    
    if len(subset) == 0:
        ax.text(0.5, 0.5, 'No data yet', ha='center', va='center', transform=ax.transAxes)
        ax.set_title(f'{dataset.upper()} - {model}')
        continue
    
    x = range(len(subset))
    baseline_line = subset['Baseline'].iloc[0]
    
    # Bar plot with error bars
    ax.bar(x, subset['FL_Accuracy'], yerr=subset['FL_Std'], capsize=5, alpha=0.7, label='FL Accuracy')
    ax.axhline(y=baseline_line, color='red', linestyle='--', linewidth=2, label='Baseline')
    
    ax.set_xticks(x)
    ax.set_xticklabels(subset['Aggregation'], rotation=45, ha='right')
    ax.set_ylabel('Accuracy (%)')
    ax.set_title(f'{dataset.upper()} - {model}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
fl_viz_path = os.path.join(models_dir, 'fl_comparison.png')
plt.savefig(fl_viz_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved: fl_comparison.png")
plt.show()

print("\n" + "="*80)
print("VISUALIZATIONS COMPLETE")
print("="*80)

## Summary

This notebook continues FL training after FedProx:

1. **Skipped**: FedAvg and FedProx (already completed)
2. **Training**: q-FedAvg, SCAFFOLD, FedAdam for all dataset/model combinations
3. **Total**: 18 remaining configurations (3 methods × 2 datasets × 2 models × 25 evaluations each)
4. **Statistical Analysis**: T-tests comparing FL against baseline with p-values
5. **Results Export**: JSON, CSV files with comprehensive statistics
6. **Visualizations**: Comparison charts showing mean ± std

Estimated runtime: 4-6 hours on Kaggle GPU.